In [ ]:
!unzip -q '/content/drive/MyDrive/oto_recommend/data/otto-validation.zip'
# !kaggle kernels output cdeotte/compute-validation-score-cv-565 -p './'
!kaggle kernels output luckyzq/otto-tuning-pipeline2-lb-0-577 -p './'

Output file downloaded to ./__notebook_source__.ipynb
Output file downloaded to ./top_50_buy2buy_fortrain_v7_0.pqt
Output file downloaded to ./top_50_carts_orders_fortrain_v7_0.pqt
Output file downloaded to ./top_50_carts_orders_fortrain_v7_1.pqt
Output file downloaded to ./top_50_carts_orders_fortrain_v7_2.pqt
Output file downloaded to ./top_50_carts_orders_fortrain_v7_3.pqt
Output file downloaded to ./top_50_clicks_fortrain_v7_0.pqt
Output file downloaded to ./top_50_clicks_fortrain_v7_1.pqt
Output file downloaded to ./top_50_clicks_fortrain_v7_2.pqt
Output file downloaded to ./top_50_clicks_fortrain_v7_3.pqt
Kernel log downloaded to ./otto-tuning-pipeline2-lb-0-577.log 


In [ ]:
## 1.召回候选数据

In [ ]:
import pandas as pd
import glob
import datetime,time
from tqdm.notebook import tqdm
from  datetime  import datetime
!pip install -q pandarallel
from pandarallel import pandarallel
import numpy as np
from tqdm import tqdm
tqdm.pandas()

  Preparing metadata (setup.py) ... done


In [ ]:
## 1.粗召回：
## (1)共现矩阵 （2）TOP XX CLICK \ CART \ ORDERS

In [ ]:
def load_test():    
    dfs = []
    for e, chunk_file in enumerate(glob.glob('/content/test_parquet/*')):
        chunk = pd.read_parquet(chunk_file)
        chunk.ts = (chunk.ts/1000).astype('int32')
        # chunk['type'] = chunk['type'].map(type_labels).astype('int8')
        dfs.append(chunk)
    return pd.concat(dfs).reset_index(drop=True) #.astype({"ts": "datetime64[ms]"})

test_df = load_test()
# test_df = pd.read_parquet('/content/test.parquet')
print('Test data has shape',test_df.shape)
test_df.head()

Test data has shape (7683577, 4)


,session,aid,ts,type
0,11819032,326879,1661354862,clicks
1,11819032,1264313,1661355133,clicks
2,11819033,840259,1661354862,clicks
3,11819033,1030009,1661365532,clicks
4,11819034,206428,1661354862,clicks


In [ ]:
%%time
VER = 7
DISK_PIECES = 4
def pqt_to_dict(df):
    return df.groupby('aid_x').aid_y.apply(list).to_dict()

# LOAD THREE CO-VISITATION MATRICES
top_20_clicks = pqt_to_dict( pd.read_parquet(f'top_50_clicks_fortrain_v{VER}_0.pqt') )

for k in range(1,DISK_PIECES): 
    top_20_clicks.update( pqt_to_dict( pd.read_parquet(f'top_50_clicks_fortrain_v{VER}_{k}.pqt') ) )


top_20_buys = pqt_to_dict( pd.read_parquet(f'top_50_carts_orders_fortrain_v{VER}_0.pqt') )

for k in range(1,DISK_PIECES): 
    top_20_buys.update( pqt_to_dict( pd.read_parquet(f'top_50_carts_orders_fortrain_v{VER}_{k}.pqt') ) )

top_20_buy2buy = pqt_to_dict( pd.read_parquet(f'top_50_buy2buy_fortrain_v{VER}_0.pqt') )

# TOP CLICKS AND ORDERS IN TEST
#top_clicks = test_df.loc[test_df['type']=='clicks','aid'].value_counts().index.values[:20]
#top_orders = test_df.loc[test_df['type']=='orders','aid'].value_counts().index.values[:20]

print('Here are size of our 3 co-visitation matrices:')
print( len( top_20_clicks ), len( top_20_buy2buy ), len( top_20_buys ) )

top_clicks = test_df.loc[test_df['type']== 'clicks','aid'].value_counts().index.values[:40] 
top_carts = test_df.loc[test_df['type']== 'carts','aid'].value_counts().index.values[:40]
top_orders = test_df.loc[test_df['type']== 'orders','aid'].value_counts().index.values[:40]

Here are size of our 3 co-visitation matrices:
1788152 1033926 1788152
CPU times: user 1min 51s, sys: 11.3 s, total: 2min 2s
Wall time: 1min 56s


In [ ]:
def get_recall_aid(x,object = 'click'):
  result = []
  if object == 'clicks':
    recall_dict = top_20_clicks
    top_index = top_clicks
  elif object == 'carts':
    recall_dict = top_20_buys
    top_index = top_carts
  elif object == 'orders':
    recall_dict = top_20_buy2buy
    top_index = top_orders

  keys = recall_dict.keys()
  for i in x:
    if i in keys:
      result += recall_dict[i]
  result = list(set(result))
  result += list(top_index)
  result += x
  return list(set(result))



type_weight_multipliers = {'clicks': 1, 'carts': 5, 'orders': 4}
def suggest_clicks(df,n):
    # USER HISTORY AIDS AND TYPES
    aids=df.aid.tolist()
    types = df.type.tolist()
    unique_aids = list(dict.fromkeys(aids[::-1] ))
    # RERANK CANDIDATES USING WEIGHTS
    if len(unique_aids)>=n:
        weights=np.logspace(0.1,1,len(aids),base=2, endpoint=True)-1
        aids_temp = Counter() 
        # RERANK BASED ON REPEAT ITEMS AND TYPE OF ITEMS
        for aid,w,t in zip(aids,weights,types): 
            aids_temp[aid] += w * type_weight_multipliers[t]
        sorted_aids = [k for k,v in aids_temp.most_common(n)]
        return sorted_aids
    # USE "CLICKS" CO-VISITATION MATRIX
    aids2 = list(itertools.chain(*[top_20_clicks[aid] for aid in unique_aids if aid in top_20_clicks]))
    # RERANK CANDIDATES
    top_aids2 = [aid2 for aid2, cnt in Counter(aids2).most_common(n) if aid2 not in unique_aids]    
    result = unique_aids + top_aids2[:n - len(unique_aids)]
    return result + list(top_clicks)[:n-len(result)]



def suggest_carts(df,n):
    # User history aids and types
    aids = df.aid.tolist()
    types = df.type.tolist()
    
    # UNIQUE AIDS AND UNIQUE BUYS
    unique_aids = list(dict.fromkeys(aids[::-1] ))
    df = df.loc[(df['type'] == 'clicks')|(df['type'] == 'carts')]
    unique_buys = list(dict.fromkeys(df.aid.tolist()[::-1]))
    
    # Rerank candidates using weights
    if len(unique_aids) >= n:
        weights=np.logspace(0.5,1,len(aids),base=2, endpoint=True)-1
        aids_temp = Counter() 
        
        # Rerank based on repeat items and types of items
        for aid,w,t in zip(aids,weights,types): 
            aids_temp[aid] += w * type_weight_multipliers[t]
        
        # Rerank candidates using"top_20_carts" co-visitation matrix
        aids2 = list(itertools.chain(*[top_20_buys[aid] for aid in unique_buys if aid in top_20_buys]))
        for aid in aids2: aids_temp[aid] += 0.1
        sorted_aids = [k for k,v in aids_temp.most_common(n)]
        return sorted_aids
    
    # Use "cart order" and "clicks" co-visitation matrices
    aids1 = list(itertools.chain(*[top_20_clicks[aid] for aid in unique_aids if aid in top_20_clicks]))
    aids2 = list(itertools.chain(*[top_20_buys[aid] for aid in unique_aids if aid in top_20_buys]))
    
    # RERANK CANDIDATES
    top_aids2 = [aid2 for aid2, cnt in Counter(aids1+aids2).most_common(n) if aid2 not in unique_aids] 
    result = unique_aids + top_aids2[:n - len(unique_aids)]
    
    # USE TOP20 TEST ORDERS
    return result + list(top_carts)[:n-len(result)]



def suggest_buys(df,n):
    # USER HISTORY AIDS AND TYPES
    aids=df.aid.tolist()
    types = df.type.tolist()
    # UNIQUE AIDS AND UNIQUE BUYS
    unique_aids = list(dict.fromkeys(aids[::-1] ))
    df = df.loc[(df['type']=='carts')|(df['type']=='orders')]
    unique_buys = list(dict.fromkeys( df.aid.tolist()[::-1] ))
    # RERANK CANDIDATES USING WEIGHTS
    if len(unique_aids)>=n:
        weights=np.logspace(0.5,1,len(aids),base=2, endpoint=True)-1
        aids_temp = Counter() 
        # RERANK BASED ON REPEAT ITEMS AND TYPE OF ITEMS
        for aid,w,t in zip(aids,weights,types): 
            aids_temp[aid] += w * type_weight_multipliers[t]
        # RERANK CANDIDATES USING "BUY2BUY" CO-VISITATION MATRIX
        aids3 = list(itertools.chain(*[top_20_buy2buy[aid] for aid in unique_buys if aid in top_20_buy2buy]))
        for aid in aids3: aids_temp[aid] += 0.1
        sorted_aids = [k for k,v in aids_temp.most_common(n)]
        return sorted_aids
    # USE "CART ORDER" CO-VISITATION MATRIX
    aids2 = list(itertools.chain(*[top_20_buys[aid] for aid in unique_aids if aid in top_20_buys]))
    # USE "BUY2BUY" CO-VISITATION MATRIX
    aids3 = list(itertools.chain(*[top_20_buy2buy[aid] for aid in unique_buys if aid in top_20_buy2buy]))
    # RERANK CANDIDATES
    top_aids2 = [aid2 for aid2, cnt in Counter(aids2+aids3).most_common(n) if aid2 not in unique_aids] 
    result = unique_aids + top_aids2[:n - len(unique_aids)]
    # USE TOP20 TEST ORDERS
    return result + list(top_orders)[:n-len(result)]

In [ ]:
import itertools
from collections import Counter
tqdm.pandas()
pandarallel.initialize(nb_workers=4,progress_bar=True)

INFO: Pandarallel will run on 4 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [ ]:
pred_df_clicks = test_df.sort_values(["session", "ts"]).groupby(["session"]).progress_apply(
    lambda x: suggest_clicks(x,40)
)

pred_df_carts = test_df.sort_values(["session", "ts"]).groupby(["session"]).progress_apply(
    lambda x: suggest_carts(x,40)
) 

pred_df_buys = test_df.sort_values(["session", "ts"]).groupby(["session"]).progress_apply(
    lambda x: suggest_buys(x,40)
)

pred_df_clicks.reset_index().rename(columns={0:'recall_aids'}).to_parquet('/content/drive/MyDrive/oto_recommend/process_data/top_40_suggest_clicks_v2.parquet')
pred_df_carts.reset_index().rename(columns={0:'recall_aids'}).to_parquet('/content/drive/MyDrive/oto_recommend/process_data/top_40_suggest_carts_v2.parquet')
pred_df_buys.reset_index().rename(columns={0:'recall_aids'}).to_parquet('/content/drive/MyDrive/oto_recommend/process_data/top_40_suggest_orders_v2.parquet')

pred_df_clicks = pd.read_parquet('/content/drive/MyDrive/oto_recommend/process_data/top_40_suggest_clicks_v2.parquet')
pred_df_carts = pd.read_parquet('/content/drive/MyDrive/oto_recommend/process_data/top_40_suggest_carts_v2.parquet')
pred_df_orders = pd.read_parquet('/content/drive/MyDrive/oto_recommend/process_data/top_40_suggest_orders_v2.parquet')

In [ ]:
test_df_1 = test_df.groupby('session')['aid'].progress_apply(lambda x:list(set(x.values))).reset_index()

100%|██████████| 1801251/1801251 [00:31<00:00, 56357.55it/s]


In [ ]:
import gc
gc.collect()

0

In [ ]:
def test_recall():
  result = {}
  score = 0
  weights = {'clicks': 0.10, 'carts': 0.30, 'orders': 0.60}
  object_list = ['clicks','carts','orders']
  for t in object_list:
    test_label = pd.read_parquet('/content/test_labels.parquet')
    test_label_tmp = test_label[test_label['type'] == t]
    test_tmp = test_df_1.merge(test_label_tmp[['session','ground_truth']],on='session')
    pred_tmp =  pd.read_parquet(f'/content/drive/MyDrive/oto_recommend/process_data/top_40_suggest_{t}_v2.parquet')
    test_tmp = test_tmp.merge(pred_tmp.rename(columns = {'recall_aids':'recall_aid'}),on='session')
    # test_tmp['recall_aid'] = test_tmp['aid'].progress_apply(lambda x : get_recall_aid(x,object=t))
    test_tmp['hits_aid'] = test_tmp.progress_apply(lambda df: list(set(df.ground_truth).intersection(df.recall_aid)), axis=1)
    test_tmp['hits'] = test_tmp['hits_aid'].apply(lambda x : len(x))
    test_tmp['gt_count'] = test_tmp.ground_truth.str.len().clip(0,20)
    recall = test_tmp['hits'].sum() / test_tmp['gt_count'].sum()
    score += weights[t]*recall
    print(f'{t} recall =',recall)
    result[t] = test_tmp
  print('=============')
  print('Overall Recall =',score)
  print('=============')
  return result

In [ ]:
result = test_recall()

100%|██████████| 1755534/1755534 [00:52<00:00, 33715.80it/s]


clicks recall = 0.5726331703060151


100%|██████████| 306341/306341 [00:08<00:00, 34237.22it/s]


carts recall = 0.44547271207080186


100%|██████████| 150817/150817 [00:04<00:00, 33231.08it/s]


orders recall = 0.6730289847208613
Overall Recall = 0.5947225214843588


In [ ]:
import gc
gc.collect()

0

In [ ]:
## 构建训练集
## 1.session, recall_aid,label是否正确,aid点击的aid

In [ ]:
## 1. 全部的
result_1 = {}
for t in ['clicks','carts','orders']:
  print('start',t)
  df_tmp = result[t]
  df_tmp_all = pd.DataFrame(np.dstack((np.repeat(df_tmp.session.values,list(map(len,df_tmp['recall_aid'].values))),np.concatenate(df_tmp['recall_aid'].values)))[0],columns=['session','recall_aids'])
  df_tmp_hist = pd.DataFrame(np.dstack((np.repeat(df_tmp.session.values,list(map(len,df_tmp['hits_aid'].values))),np.concatenate(df_tmp['hits_aid'].values)))[0],columns=['session','hits_aids'])
  df_tmp= df_tmp_all.merge(df_tmp_hist,left_on=['session','recall_aids'],right_on=['session','hits_aids'],how= 'left')
  del df_tmp_all,df_tmp_hist
  gc.collect()
  df_tmp['label'] = 1
  df_tmp.loc[df_tmp['hits_aids'].isna(),'label'] = 0
  result_1[t] = df_tmp
  gc.collect()

start clicks
start carts
start orders


In [ ]:
result_1['clicks'].to_parquet('/content/drive/MyDrive/oto_recommend/process_data/train_clicks_v2.parquet',index=False)
result_1['carts'].to_parquet('/content/drive/MyDrive/oto_recommend/process_data/train_carts_v2.parquet',index=False)
result_1['orders'].to_parquet('/content/drive/MyDrive/oto_recommend/process_data/train_orders_v2.parquet',index=False)